# <span style="color:green">       _________ ***PROYECTO FINAL:***


##  <span style="color:green"> ***ANIME 1: ESTIMACION DE RATINGS EN ANIMES.***


### Carrera: Data Science
### Academía: DesafioLatam https://desafiolatam.com/
### Lugar: Santiago de Chile.
### Generación: G56
### Fecha:  29/06/2023

##  <span style="color:green"> ***Descripción general:***
###   En este Proyecto trabajamos con datos de la plataforma web dedicada a la distribución de información sobre valoraciones de usuarios para series y películas de Anime está enfrentando un problema. Lamentablemente, un porcentaje de la muestra no tiene valoración, por lo tanto, se desea crear un sistema que ayude a estimar el puntaje que una serie/película debería tener dadas sus características. Es importante destacar que dicha herramienta o sistema será el principal componente de una mejora futura que podría ser comercializada. 

###    Para lograr lo planteado, se trabajará en cinco (5) Secciones que a continuación se enumerán (mismo orden del desarrollo):

####    1.- Resumen.
####    2.- Aspectos Computacionales.
####    3.- Ingenieria de Datos.          ...    (Ojo: podría ser "Ingenieria de Datos/Analisis Exploratorios")
####    4.- Generar matrices: "X" e  "y"  ...      (Ojo: podría ser "Feature Engineering") .
####    5.- Modelos a utilizar            ...   (Ojo: podría ser Modelación )


.

##  <span style="color:green"> ***1.- Resumen***

### **Requerimiento general**: Crear un sistema que ayude a estimar el puntaje que una serie/película debería tener dadas sus características.

***Vector Objetivo***: Variable 'rating'.

***Data set usado en el modelo***: df_anime.

1. anime.csv (caracteristicas de los animes)
    * anime_id: Identificador único del anime.
    * name: Nombre completo del anime.
    * genre: Lista de géneros (separados por coma) en el que califica el anime.
    * type: Tipo de presentación del anime, puede ser: movie, TV, OVA, etc.
    * episodes: Número de episodios del anime (1 para el caso películas).
    * rating: Evaluación promedio del anime.
    * members: Número de miembros de la comunidad que están en el “grupo” del anime.

#  <span style="color:green"> ***2.- Aspectos computacionales***

### 2.1 .- Versiones

### 2.2 .- Librerias

In [1]:
import pandas as pd
import numpy as np 
import pickle as p

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

#imputar datos
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score, confusion_matrix

from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor


### 2.3 Funciones Creadas

###  OJO**/OJO   ****** (Comentario nuestro: se hará y anexará un archivo de funciones.....parte del entregable..???)
###                               Sin embargo, en este punto (2.3) colocaríamos solo el nombre de las funciones..?????

In [2]:

def estudio_nulos(data, simbolo):
    ''' 
    [resumen]: Imprime cantidad de perdidos en cada columna y devuelve listas,
               todo en base al simbolo proporcionado
    [data]: DataFrame
    [simbolo]: Caracter, string, valor a buscar como nulo o perdido
    [return]: lista_sincaract, lista_concaract
    '''
    lista_sinvacios = []
    lista_convacios = []
    for i in data.columns:
        try:
            print(f'{i}: {data[i].value_counts()[simbolo]}')
            lista_convacios.append(i)
        except:
            lista_sinvacios.append(i)
    return lista_sinvacios, lista_convacios

#conteo de np.nan
def contar_nan(data, lista_nulos):
    ''' 
    Ingresa una data y una lista de columnas
    para imprimir los % de nan por columnas, y el total porlineas       
    '''
    for col in lista_nulos:
        nulos = data[col].isnull().value_counts("%")
        try:
            nulos = nulos[True]*100
            print(f'{col}: {np.round(nulos,4)}%\n')
        except:
            print(f'{col}: sin nulos')
    tmp_drop = data.dropna().copy()
    porcion_nulos = (1-(tmp_drop.shape[0]/data.shape[0]))*100
    print(f'% de lineas con nan en la data {np.round(porcion_nulos,4)}%')

#imputar datos perdidos
def entrenar_imputador(data:pd, strategy:str,dev_matriz=False, missing_val=np.nan, tamaño_test=0.33, random=1991):
    ''' 
    Input:
    data:[pd.Series]: Base de datos o vector
    strategy:[str]: Estrategias permitidas por SimpleImputer de Sklearn
    missing_val=np.nan [opc]
    dev_matriz=False [opc]
    tamaño_test=0.33 [opc]
    random=1991 [opc]
    Return:
    imp: Modelo entrenado a partir de sub matriz para reemplazar missing_val
    x_train: Sub matriz de entrenamiento
    x_test: Sub matriz de prueba
    df_clean: Base de datos sin valores perdidos.
    '''
    x_train, x_test = train_test_split(data, test_size=tamaño_test, random_state=random)
    imp = SimpleImputer(missing_values=missing_val, strategy=strategy)
    imp.fit_transform(x_train)
    df_tmp = data.copy()
    df_clean = pd.DataFrame(imp.transform(df_tmp), columns=data.columns)
    if dev_matriz:
        return imp, x_train, x_test, df_clean
    else:
        return imp, df_clean
    
def probar_modelos(modelo, X, y):
    '''
    [resumen]: modela, predice e imprime resultados MSE y R2
    [modelo]: modelo de sklearn
    [X]: matriz de atributos
    [y]: Vector objectivo
    [return]: none 
    '''
    modelo_fit = modelo.fit(X,y)
    y_hat = modelo_fit.predict(X)

    print(f'''{modelo}
MAE: {mean_absolute_error(y_true=y, y_pred=y_hat)}
R2: {r2_score(y_true=y, y_pred=y_hat)}
---------------------------------------------------''')

#  <span style="color:green"> ***3.- Ingenieria de Datos***

1. Imputar valores nulos con la estrategia **mas frecuente** 
2. Corregir las palabras mal escritas de la columna genre.
3. Aplicar dummies en las columnas genere y type.

## 3.1 .- Importar del dataset "anime" y construcción del dataframe: df

In [7]:
#         Importación de datos al ambiente de trabajo. 
df= pd.read_csv('/Users/elio1/Dataset_csv/anime.csv', delimiter=';') # Descg. archivo asignado.
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,"9,37",200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,"9,26",793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,"9,25",114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,"9,17",673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,"9,16",151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,"4,15",211
12290,5543,Under World,Hentai,OVA,1,"4,28",183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,"4,88",219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,"4,98",175


## 3.2 .- Exploración y tratamiento Nulos, vacios y valores especiales

### Columna 'name'

In [18]:
df['name'].value_counts()

Shi Wan Ge Leng Xiaohua                           2
Saru Kani Gassen                                  2
Bakabon Osomatsu no Karee wo Tazunete Sansenri    1
Backkom Meogeujan Yeohaeng                        1
Backkom Mission Impossible                        1
                                                 ..
Yoroiden Samurai Troopers Kikoutei Densetsu       1
Yuu☆Yuu☆Hakusho: Mu Mu Hakusho                    1
3-gatsu no Lion meets Bump of Chicken             1
Bannou Bunka Neko-Musume                          1
Yasuji no Pornorama: Yacchimae!!                  1
Name: name, Length: 12292, dtype: int64

In [8]:
df.loc[df['name'] =='Shi Wan Ge Leng Xiaohua']

,anime_id,name,genre,type,episodes,rating,members
10193,33193,Shi Wan Ge Leng Xiaohua,"Comedy, Parody",ONA,12,"6,67",114
10194,33195,Shi Wan Ge Leng Xiaohua,"Action, Adventure, Comedy, Fantasy, Parody",Movie,1,"7,07",110


In [9]:
df.loc[df['name'] =='Saru Kani Gassen']

,anime_id,name,genre,type,episodes,rating,members
10140,22399,Saru Kani Gassen,Kids,OVA,1,"5,23",62
10141,30059,Saru Kani Gassen,Drama,Movie,1,"4,75",76


Son nombres repetidos, sin embargo apuntan a sus respectivos tipos distintos, por ende se dejan ambos

### Columna 'genre'

Eliminación de caracteres especiales:

In [10]:
df['genre'] = df['genre'].str.replace(', ', ',')
df['genre'] = df['genre'].str.replace('Sclice of Life', 'Slice of Life')
df['genre'] = df['genre'].str.replace('-', '_')
df['genre'] = df['genre'].str.replace(' ', '_')

### Columna 'Episode'

In [11]:
df['episodes'].isna().value_counts()

False    12294
Name: episodes, dtype: int64

In [12]:
df['episodes'][df['episodes']== 'Unknown'].value_counts()

Unknown    340
Name: episodes, dtype: int64

In [13]:
df['episodes'] = np.where(df['episodes'] == 'Unknown', np.nan, df['episodes'])

### Columna 'Rating'

In [14]:
df['rating'] = df['rating'].str.replace(',', '.').astype(float)

## 3.3 .- Analisis de np.nan en "df_anime"

### Imputar np.nan

In [15]:
contar_nan(df, df.columns)

anime_id: sin nulos
name: sin nulos
genre: 0.3823%

type: 0.2034%

episodes: 2.7656%

rating: 1.8708%

members: sin nulos
% de lineas con nan en la data 3.7742%


In [16]:
imputer, df_clean = entrenar_imputador(df.drop(columns='rating'), strategy='most_frequent',dev_matriz=False, missing_val=np.nan, tamaño_test=0.33, random=1991)

In [17]:
df_clean = pd.concat([df_clean,df['rating']], axis=1)
df_clean.head(5)

,anime_id,name,genre,type,episodes,members,rating
0,32281,Kimi no Na wa.,"Drama,Romance,School,Supernatural",Movie,1,200630,9.37
1,5114,Fullmetal Alchemist: Brotherhood,"Action,Adventure,Drama,Fantasy,Magic,Military,...",TV,64,793665,9.26
2,28977,Gintama°,"Action,Comedy,Historical,Parody,Samurai,Sci_Fi...",TV,51,114262,9.25
3,9253,Steins;Gate,"Sci_Fi,Thriller",TV,24,673572,9.17
4,9969,Gintama&#039;,"Action,Comedy,Historical,Parody,Samurai,Sci_Fi...",TV,51,151266,9.16


In [18]:
contar_nan(df_clean, df.columns)

anime_id: sin nulos
name: sin nulos
genre: sin nulos
type: sin nulos
episodes: sin nulos
rating: 1.8708%

members: sin nulos
% de lineas con nan en la data 1.8708%


### Separar matriz de nulos en el vector objetivo 'rating'

In [19]:
df_test_nulos = df_clean.loc[df_clean['rating'].isna()]

In [20]:
df_clean = df_clean.dropna()

In [21]:
contar_nan(df_clean, df.columns)

anime_id: sin nulos
name: sin nulos
genre: sin nulos
type: sin nulos
episodes: sin nulos
rating: sin nulos
members: sin nulos
% de lineas con nan en la data 0.0%


In [22]:
df_clean.shape

(12064, 7)

## 3.4 Recodificar columna 'genre'

In [23]:
(df_clean['genre'] == '').value_counts()

False    12064
Name: genre, dtype: int64

#  <span style="color:green"> ***4 .- Generar matrices: "X" e  "y"***

## 4.1 .- Dummies

In [24]:
df_modelo = pd.get_dummies(df_clean[['genre','type']])
df_modelo['episodes'] = df_clean['episodes'].astype(int).copy()
df_modelo['members'] = df_clean['members'].astype(int).copy()
df_modelo.head(5)

,genre_Action,"genre_Action,Adventure","genre_Action,Adventure,Cars,Comedy,Sci_Fi,Shounen","genre_Action,Adventure,Cars,Mecha,Sci_Fi,Shounen,Sports","genre_Action,Adventure,Cars,Sci_Fi","genre_Action,Adventure,Comedy","genre_Action,Adventure,Comedy,Demons,Drama,Ecchi,Horror,Mystery,Romance,Sci_Fi","genre_Action,Adventure,Comedy,Demons,Fantasy,Magic","genre_Action,Adventure,Comedy,Demons,Fantasy,Magic,Romance,Shounen,Supernatural","genre_Action,Adventure,Comedy,Demons,Fantasy,Martial_Arts,Shounen,Super_Power",...,genre_Vampire,genre_Yaoi,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,episodes,members
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,200630
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,64,793665
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,51,114262
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,24,673572
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,51,151266


In [25]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12064 entries, 0 to 12293
Columns: 3237 entries, genre_Action to members
dtypes: int32(2), uint8(3235)
memory usage: 37.4 MB


In [26]:
df_modelo.shape

(12064, 3237)

In [27]:
y = df_clean['rating']

##  <span style="color:green"> ***5.- Modelos a utilizar*** 

### 5.1 .- Iteracion sin hiper parametros

In [35]:
modelos_lista = [
    ElasticNetCV(),
    LinearRegression()
    #SVR(),
    #DecisionTreeRegressor(),
    #RandomForestRegressor(),  
    #AdaBoostRegressor(),
    #GradientBoostingRegressor()
]

In [36]:
# Función para generar los Reportes de los 7 modelos:
def probar_modelos(modelo, X, y):
    modelo_fit = modelo.fit(X,y)
    y_hat = modelo_fit.predict(X)

    print(f'''
    Model: {modelo}
    MAE: {mean_absolute_error(y_true=y, y_pred=y_hat).round(4)}
    R2: {r2_score(y_true=y, y_pred=y_hat).round(4)}
    --------------------------------------------''')
   

In [37]:
for modelo in modelos_lista:
    probar_modelos(modelo, df_modelo,y)

###   Tiempo de ejecución estimado (todos los 7 modelos) = 15 minutos


Model: ElasticNetCV()
MAE: 0.732
R2: 0.15
---------------------------------------------------

Model: LinearRegression()
MAE: 0.459
R2: 0.567
---------------------------------------------------


In [90]:
estimadores = [
        ('lr',LinearRegression()),
        ('DT',DecisionTreeRegressor()),
        ('RF',RandomForestRegressor()),
        ('GB',GradientBoostingRegressor())
        ]

In [91]:
modelo_voting = VotingRegressor(estimators=estimadores,
                                verbose=True,
                                n_jobs=-1
                                )

In [92]:
probar_modelos(modelo_voting, df_modelo,y)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('DT', DecisionTreeRegressor()),
                            ('RF', RandomForestRegressor()),
                            ('GB', GradientBoostingRegressor())],
                n_jobs=-1, verbose=True)
MAE: 0.27029151975681937
R2: 0.8611765840851258
---------------------------------------------------


## 5.2 .- Modelos seleccionados

* LinearRegression()
* DecisionTreeRegressor()
* RandomForestRegressor()
* GradientBoostingRegressor()
* VotingRegressor()

# NOTAS

In [101]:
min(df_clean['rating'].value_counts().index)

1.67

In [102]:
max(df_clean['rating'].value_counts().index)

10.0

mae no mayor a 10
r2 mayor a 10 
rmse comparar con otros modelos. 

5.X .- Calculo del Código del "Information Value" (IV)

In [ ]:
def calculate_iv(df, feature, target):
    """
    Calculate the information value (IV) of a feature in a dataset.
    
    Parameters:
    df (pandas.DataFrame): the dataset
    feature (str): the name of the feature to calculate IV for
    target (str): the name of the target variable
    
    Returns:
    float: the information value (IV) of the feature
    """
    df = df[[feature, target]]
    df = df.dropna()
    n = df.shape[0]
    good = df[target].sum()
    bad = n - good
    unique_values = df[feature].unique()
    iv = 0
    for value in unique_values:
        n1 = df[df[feature] == value].shape[0]
        good1 = df[(df[feature] == value) & (df[target] == 1)].shape[0]
        bad1 = n1 - good1
        if good1 == 0 or bad1 == 0:
            continue
        woe = np.log((good1 / good) / (bad1 / bad))
        iv += (good1 / good - bad1 / bad) * woe
    return iv

#  <span style="color:green"> ***Comentarios finales:*** </span>